# Lab 3: Quote Function Tool

In this lab, you will add a second Azure Function tool (quote of the day) and see how the agent handles multiple tools.

## Learning Objectives

- Register multiple tools with an agent
- Understand tool selection by the AI
- Handle tools with parameters
- Create more complex interactions

## Prerequisites

- Completed Labs 1 and 2
- Azure Functions deployed with both `/health` and `/quote` endpoints

## Step 1: Import Required Modules

In [ ]:
from northwestern_foundry_agent import Settings, FoundryAgent
from northwestern_foundry_agent.foundry.tools import (
    create_health_check_tool,
    create_quote_tool,
)
from northwestern_foundry_agent.integrations.azure_functions import (
    AzureFunctionsClient,
    create_function_handlers,
)
from northwestern_foundry_agent.utils import setup_logging, get_logger

setup_logging(Settings(log_level="INFO", log_format="text"))
logger = get_logger(__name__)

print("Modules imported successfully!")

## Step 2: Load Configuration

In [ ]:
settings = Settings()

print(f"Azure AI configured: {settings.is_configured}")
print(f"Functions configured: {settings.functions_configured}")

## Step 3: Test Quote Function Directly

In [ ]:
if settings.functions_configured:
    try:
        client = AzureFunctionsClient(settings)
        
        # Test with different categories
        for category in ["motivation", "wisdom", "humor"]:
            quote = await client.quote_of_the_day(category=category)
            print(f"\n{category.upper()}:")
            print(f'  "{quote.quote}"')
            print(f"  - {quote.author}")
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Functions not configured. Example quotes:")
    print('\nMOTIVATION:')
    print('  "The only way to do great work is to love what you do."')
    print('  - Steve Jobs')
    print('\nWISDOM:')
    print('  "The only true wisdom is in knowing you know nothing."')
    print('  - Socrates')

## Step 4: View the Quote Tool Definition

In [ ]:
import json

quote_endpoint = f"{settings.azure_function_app_url}/api/quote" if settings.functions_configured else "https://example.azurewebsites.net/api/quote"
quote_tool = create_quote_tool(quote_endpoint)

print("Quote Tool Definition:")
print(f"  Name: {quote_tool.name}")
print(f"  Description: {quote_tool.description}")

print("\nParameters:")
for param in quote_tool.parameters:
    print(f"  - {param.name}: {param.param_type}")
    print(f"    Required: {param.required}")
    print(f"    Options: {param.enum}")
    print(f"    Default: {param.default}")

print("\nFunction Definition (JSON):")
print(json.dumps(quote_tool.to_function_definition(), indent=2))

## Step 5: Create Agent with Multiple Tools

In [ ]:
AGENT_INSTRUCTIONS = """
You are a helpful assistant with access to system tools.

You have the following tools available:

1. **health_check**: Use this to check system health status.
   - No parameters required
   - Returns: status, service name, version, timestamp

2. **quote_of_the_day**: Use this to get inspirational quotes.
   - Parameter: category (motivation, wisdom, or humor)
   - Returns: quote text, author, category

Guidelines:
- When asked about health/status, use health_check
- When asked for quotes/inspiration, use quote_of_the_day
- If the user specifies a quote category, use that category
- If no category specified, default to motivation
- Present quotes in a nice format with attribution
"""

if settings.is_configured and settings.functions_configured:
    try:
        agent = FoundryAgent(settings)
        func_client = AzureFunctionsClient(settings)
        handlers = create_function_handlers(func_client)
        
        # Create tool definitions
        health_endpoint = f"{settings.azure_function_app_url}/api/health"
        health_tool = create_health_check_tool(health_endpoint)
        
        # Register both tools
        agent.register_tool(health_tool, handlers["health_check"])
        agent.register_tool(quote_tool, handlers["quote_of_the_day"])
        
        # Create agent
        created_agent = agent.create_agent(
            name="multi-tool-agent",
            instructions=AGENT_INSTRUCTIONS,
        )
        
        print(f"Agent created with multiple tools!")
        print(f"Registered tools: {list(agent.tools.keys())}")
    except Exception as e:
        print(f"Error: {e}")
        agent = None
else:
    print("Skipping (Azure not fully configured)")
    agent = None

## Step 6: Test Multi-Tool Agent

Let's try different queries and see how the agent selects the appropriate tool.

In [ ]:
# Test queries that should trigger different tools
test_queries = [
    "What's the system health status?",
    "Give me a motivational quote",
    "I need some wisdom for today",
    "Tell me something funny",
    "Check the system and also give me an inspiring quote",
]

if agent and agent.agent:
    for query in test_queries:
        print(f"\n{'='*60}")
        print(f"User: {query}")
        print(f"{'='*60}")
        try:
            response = agent.run(query)
            print(f"\nAgent: {response}")
        except Exception as e:
            print(f"Error: {e}")
else:
    print("Agent not available. Example interactions:")
    print("\n" + "="*60)
    print("User: Give me a motivational quote")
    print("="*60)
    print("\n[Agent calls quote_of_the_day(category='motivation')]")
    print('\nAgent: Here\'s an inspiring quote for you:')
    print('\n  "The only way to do great work is to love what you do."')
    print('  — Steve Jobs')

## Step 7: Clean Up

In [ ]:
if agent and agent.agent:
    try:
        agent.delete_agent()
        print("Agent deleted successfully!")
    except Exception as e:
        print(f"Error: {e}")

## Summary

In this lab, you learned how to:

1. ✅ Register multiple tools with an agent
2. ✅ Understand tool definitions with parameters
3. ✅ Observe how the AI selects appropriate tools based on queries
4. ✅ Handle tools that accept optional parameters

## Key Observations

- The agent automatically selects the right tool based on the query
- Parameters are inferred from the user's request (e.g., "wisdom" → category="wisdom")
- Agents can use multiple tools in a single response

## Next Steps

Continue to **Lab 4: Logic App Integration** to learn how to integrate Azure Logic Apps as tools.